# 第7课：模型持久化与部署

## 学习目标
- 掌握模型保存和加载方法
- 了解模型部署的基本流程
- 学会使用 Flask 创建 API
- 了解模型监控和维护

## 1. 模型持久化

In [ ]:
import numpy as np
import pandas as pd
import pickle
import joblib
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# 加载数据并训练模型
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# 创建流水线
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline.fit(X_train, y_train)
print(f"模型训练完成，测试准确率: {pipeline.score(X_test, y_test):.3f}")

In [ ]:
# 1.1 使用 pickle 保存模型
with open('model_pickle.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

# 加载模型
with open('model_pickle.pkl', 'rb') as f:
    loaded_model_pickle = pickle.load(f)

print(f"Pickle 加载模型准确率: {loaded_model_pickle.score(X_test, y_test):.3f}")

In [ ]:
# 1.2 使用 joblib 保存模型（推荐用于大模型）
joblib.dump(pipeline, 'model_joblib.pkl')

# 加载模型
loaded_model_joblib = joblib.load('model_joblib.pkl')

print(f"Joblib 加载模型准确率: {loaded_model_joblib.score(X_test, y_test):.3f}")

In [ ]:
# 1.3 保存模型元数据
import json
from datetime import datetime

model_metadata = {
    'model_name': 'iris_classifier',
    'model_type': 'RandomForestClassifier',
    'version': '1.0.0',
    'training_date': datetime.now().isoformat(),
    'features': iris.feature_names.tolist(),
    'target_names': iris.target_names.tolist(),
    'accuracy': float(pipeline.score(X_test, y_test)),
    'hyperparameters': {
        'n_estimators': 100,
        'random_state': 42
    }
}

with open('model_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(model_metadata, f, indent=2, ensure_ascii=False)

print("模型元数据:")
print(json.dumps(model_metadata, indent=2, ensure_ascii=False))

## 2. 模型版本管理

In [ ]:
import os
from datetime import datetime

class ModelVersionManager:
    def __init__(self, base_path='models'):
        self.base_path = base_path
        os.makedirs(base_path, exist_ok=True)
    
    def save_model(self, model, model_name, metrics=None):
        """保存模型并创建版本"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        version_path = os.path.join(self.base_path, model_name, timestamp)
        os.makedirs(version_path, exist_ok=True)
        
        # 保存模型
        model_path = os.path.join(version_path, 'model.pkl')
        joblib.dump(model, model_path)
        
        # 保存元数据
        metadata = {
            'model_name': model_name,
            'version': timestamp,
            'created_at': datetime.now().isoformat(),
            'metrics': metrics or {}
        }
        metadata_path = os.path.join(version_path, 'metadata.json')
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        
        print(f"模型已保存: {version_path}")
        return version_path
    
    def load_model(self, model_name, version='latest'):
        """加载指定版本的模型"""
        model_dir = os.path.join(self.base_path, model_name)
        
        if version == 'latest':
            versions = sorted(os.listdir(model_dir))
            version = versions[-1]
        
        model_path = os.path.join(model_dir, version, 'model.pkl')
        return joblib.load(model_path)
    
    def list_versions(self, model_name):
        """列出所有版本"""
        model_dir = os.path.join(self.base_path, model_name)
        if os.path.exists(model_dir):
            return sorted(os.listdir(model_dir))
        return []

# 使用示例
manager = ModelVersionManager()

# 保存模型
metrics = {'accuracy': 0.95, 'f1_score': 0.94}
manager.save_model(pipeline, 'iris_classifier', metrics)

# 列出版本
print(f"\n可用版本: {manager.list_versions('iris_classifier')}")

## 3. 创建预测 API

In [ ]:
# 创建 Flask API（保存为 app.py）
flask_code = '''
from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# 加载模型
model = joblib.load("model_joblib.pkl")

# 特征名和类别名
feature_names = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
target_names = ["setosa", "versicolor", "virginica"]

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "healthy"})

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.json
        features = np.array([[data[f] for f in feature_names]])
        
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0].tolist()
        
        return jsonify({
            "prediction": target_names[prediction],
            "prediction_id": int(prediction),
            "probabilities": dict(zip(target_names, probability))
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 400

@app.route("/batch_predict", methods=["POST"])
def batch_predict():
    try:
        data = request.json
        samples = data["samples"]
        features = np.array([[s[f] for f in feature_names] for s in samples])
        
        predictions = model.predict(features)
        
        results = [
            {"prediction": target_names[p], "prediction_id": int(p)}
            for p in predictions
        ]
        
        return jsonify({"predictions": results})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)
'''

print("Flask API 代码示例:")
print(flask_code)

In [ ]:
# 保存 Flask 应用代码
with open('app.py', 'w', encoding='utf-8') as f:
    f.write(flask_code)

print("Flask 应用已保存到 app.py")
print("\n运行命令: python app.py")
print("\n测试 API:")
print('''curl -X POST http://localhost:5000/predict \
     -H "Content-Type: application/json" \
     -d '{"sepal_length": 5.1, "sepal_width": 3.5, "petal_length": 1.4, "petal_width": 0.2}' ''')

## 4. 使用 FastAPI（现代方案）

In [ ]:
fastapi_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import joblib
import numpy as np

app = FastAPI(title="Iris Classifier API", version="1.0.0")

# 加载模型
model = joblib.load("model_joblib.pkl")
target_names = ["setosa", "versicolor", "virginica"]

# 请求模型
class IrisFeatures(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

class BatchRequest(BaseModel):
    samples: List[IrisFeatures]

# 响应模型
class PredictionResponse(BaseModel):
    prediction: str
    prediction_id: int
    probabilities: dict

@app.get("/health")
def health():
    return {"status": "healthy"}

@app.post("/predict", response_model=PredictionResponse)
def predict(features: IrisFeatures):
    try:
        X = np.array([[features.sepal_length, features.sepal_width,
                       features.petal_length, features.petal_width]])
        
        prediction = model.predict(X)[0]
        probability = model.predict_proba(X)[0].tolist()
        
        return {
            "prediction": target_names[prediction],
            "prediction_id": int(prediction),
            "probabilities": dict(zip(target_names, probability))
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.post("/batch_predict")
def batch_predict(batch: BatchRequest):
    try:
        X = np.array([[s.sepal_length, s.sepal_width, 
                       s.petal_length, s.petal_width] 
                      for s in batch.samples])
        
        predictions = model.predict(X)
        
        return {
            "predictions": [
                {"prediction": target_names[p], "prediction_id": int(p)}
                for p in predictions
            ]
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# 运行: uvicorn fastapi_app:app --reload
'''

with open('fastapi_app.py', 'w', encoding='utf-8') as f:
    f.write(fastapi_code)

print("FastAPI 应用已保存到 fastapi_app.py")
print("\n运行命令: uvicorn fastapi_app:app --reload")
print("API 文档: http://localhost:8000/docs")

## 5. Docker 容器化

In [ ]:
# Dockerfile
dockerfile = '''
FROM python:3.10-slim

WORKDIR /app

# 安装依赖
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 复制代码和模型
COPY app.py .
COPY model_joblib.pkl .

# 暴露端口
EXPOSE 5000

# 启动应用
CMD ["python", "app.py"]
'''

# requirements.txt
requirements = '''
flask==2.3.0
scikit-learn==1.3.0
joblib==1.3.0
numpy==1.24.0
'''

print("Dockerfile:")
print(dockerfile)
print("\nrequirements.txt:")
print(requirements)

# 保存文件
with open('Dockerfile', 'w', encoding='utf-8') as f:
    f.write(dockerfile)

with open('requirements.txt', 'w', encoding='utf-8') as f:
    f.write(requirements)

print("\n构建 Docker 镜像: docker build -t iris-classifier .")
print("运行容器: docker run -p 5000:5000 iris-classifier")

## 6. 模型监控

In [ ]:
import logging
from datetime import datetime
import json

class ModelMonitor:
    def __init__(self, model_name):
        self.model_name = model_name
        self.predictions = []
        
        # 设置日志
        logging.basicConfig(
            filename=f'{model_name}_predictions.log',
            level=logging.INFO,
            format='%(asctime)s - %(message)s'
        )
    
    def log_prediction(self, input_features, prediction, probability=None):
        """记录预测"""
        record = {
            'timestamp': datetime.now().isoformat(),
            'input': input_features.tolist() if hasattr(input_features, 'tolist') else input_features,
            'prediction': prediction,
            'probability': probability
        }
        self.predictions.append(record)
        logging.info(json.dumps(record))
    
    def get_statistics(self):
        """获取预测统计"""
        if not self.predictions:
            return {}
        
        predictions_list = [p['prediction'] for p in self.predictions]
        from collections import Counter
        counts = Counter(predictions_list)
        
        return {
            'total_predictions': len(self.predictions),
            'prediction_distribution': dict(counts),
            'first_prediction': self.predictions[0]['timestamp'],
            'last_prediction': self.predictions[-1]['timestamp']
        }

# 使用示例
monitor = ModelMonitor('iris_classifier')

# 模拟预测并记录
for i in range(10):
    X_sample = X_test[i:i+1]
    pred = pipeline.predict(X_sample)[0]
    prob = pipeline.predict_proba(X_sample)[0].tolist()
    monitor.log_prediction(X_sample[0], iris.target_names[pred], prob)

print("预测统计:")
print(json.dumps(monitor.get_statistics(), indent=2))

In [ ]:
# 数据漂移检测
from scipy import stats

def detect_drift(reference_data, current_data, threshold=0.05):
    """使用 KS 检验检测数据漂移"""
    drift_results = {}
    
    for i, feature in enumerate(iris.feature_names):
        ref = reference_data[:, i]
        cur = current_data[:, i]
        
        statistic, p_value = stats.ks_2samp(ref, cur)
        drift_results[feature] = {
            'statistic': statistic,
            'p_value': p_value,
            'drift_detected': p_value < threshold
        }
    
    return drift_results

# 模拟新数据（带漂移）
np.random.seed(123)
X_new = X_test + np.random.normal(0.5, 0.1, X_test.shape)

drift_results = detect_drift(X_train, X_new)
print("数据漂移检测结果:")
for feature, result in drift_results.items():
    status = "检测到漂移" if result['drift_detected'] else "正常"
    print(f"  {feature}: p={result['p_value']:.4f} - {status}")

## 7. 练习题

### 练习：创建完整的模型部署流程

In [ ]:
# 在这里编写代码
# 1. 训练一个分类模型
# 2. 保存模型和元数据
# 3. 创建预测函数
# 4. 添加日志记录
# 5. 实现简单的监控


## 8. 本课小结

1. **模型持久化**：pickle、joblib 保存和加载
2. **版本管理**：记录模型版本和元数据
3. **API 部署**：Flask、FastAPI 创建 REST API
4. **容器化**：使用 Docker 打包部署
5. **模型监控**：记录预测、检测数据漂移

## 9. 清理临时文件

In [ ]:
# 清理示例中创建的文件（可选）
import os

files_to_clean = [
    'model_pickle.pkl', 'model_joblib.pkl', 'model_metadata.json',
    'app.py', 'fastapi_app.py', 'Dockerfile', 'requirements.txt',
    'iris_classifier_predictions.log'
]

# 取消注释以下代码来清理文件
# for f in files_to_clean:
#     if os.path.exists(f):
#         os.remove(f)
#         print(f"已删除: {f}")

print("提示：如需清理临时文件，请取消上方代码的注释")